# BI - Laboratorio 1
Jose David Martinez - 202116677
Manuel Felipe Carvajal - 202014203
Juanita Gil - 



# 0 - Imports

In [2]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Librerías de preprocesamiento y NLP
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Librerías de scikit-learn para modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score


# 1. Carga de Datos

In [3]:
# Cargar el dataset de opiniones
data = pd.read_excel("Datos_proyecto.xlsx")

# Revisar primeras filas
data.head()

# Información general
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


# 2. Entendimiento y preparación de datos

En primer lugar se analizó la distribución de clases presentes en el dataset. Este análisis permite identificar cuántas opiniones corresponden a cada uno de los Objetivos de Desarrollo Sostenible considerados en el proyecto: ODS 1 (Fin de la pobreza), ODS 3 (Salud y bienestar) y ODS 4 (Educación de calidad). Los resultados evidencian la cantidad de registros por categoría y permiten observar posibles desbalances entre clases, lo cual es un factor relevante porque puede afectar el desempeño de los modelos de clasificación. Este paso inicial es fundamental para conocer la representatividad de los datos y orientar las decisiones posteriores en cuanto a técnicas de preprocesamiento y modelado.